In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from datetime import datetime, timedelta
import time

import gc
import copy

import pyarrow.parquet as pq
import pyarrow as pa

 
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [3]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
random_seed=8968
pl.seed_everything(random_seed)

Global seed set to 8968


8968

In [4]:
%%time
train_file = r'amex\agg_train_all_rev.parquet'
df=pd.read_parquet(train_file, engine='pyarrow')

Wall time: 4.59 s


In [5]:
all_cols = ['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_31=0', 'B_31=1', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38=1.0', 'B_38=2.0', 'B_38=3.0', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_114=0.0', 'D_114=1.0', 'D_115', 'D_116=0.0', 'D_116=1.0', 'D_117=-1.0', 'D_117=1.0', 'D_117=2.0', 'D_117=3.0', 'D_117=4.0', 'D_117=5.0', 'D_117=6.0', 'D_118', 'D_119', 'D_120=0.0', 'D_120=1.0', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126=-1.0', 'D_126=0.0', 'D_126=1.0', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63=CL', 'D_63=CO', 'D_63=CR', 'D_63=XL', 'D_63=XM', 'D_63=XZ', 'D_64=-1', 'D_64=O', 'D_64=R', 'D_64=U', 'D_65', 'D_66=0.0', 'D_66=1.0', 'D_68=0.0', 'D_68=1.0', 'D_68=2.0', 'D_68=3.0', 'D_68=4.0', 'D_68=5.0', 'D_68=6.0', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96', 'P_2', 'P_3', 'P_4', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 'R_21', 'R_22', 'R_23', 'R_24', 'R_25', 'R_26', 'R_27', 'R_28', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_2=max', 'S_2=min', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'customer_ID', 'days']

cat_feats = ['B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_31=0', 'B_31=1', 'B_38=1.0', 'B_38=2.0', 'B_38=3.0', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'D_114=0.0', 'D_114=1.0', 'D_116=0.0', 'D_116=1.0', 'D_117=-1.0', 'D_117=1.0', 'D_117=2.0', 'D_117=3.0', 'D_117=4.0', 'D_117=5.0', 'D_117=6.0', 'D_120=0.0', 'D_120=1.0', 'D_126=-1.0', 'D_126=0.0', 'D_126=1.0', 'D_63=CL', 'D_63=CO', 'D_63=CR', 'D_63=XL', 'D_63=XM', 'D_63=XZ', 'D_64=-1', 'D_64=O', 'D_64=R', 'D_64=U', 'D_66=0.0', 'D_66=1.0', 'D_68=0.0', 'D_68=1.0', 'D_68=2.0', 'D_68=3.0', 'D_68=4.0', 'D_68=5.0', 'D_68=6.0']
s2_feats = ['S_2=max', 'S_2=min']
float_feats = ['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_32', 'B_33', 'B_36', 'B_37', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96', 'P_2', 'P_3', 'P_4', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 'R_21', 'R_22', 'R_23', 'R_24', 'R_25', 'R_26', 'R_27', 'R_28', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'days']
log_feats = ['log_B_11', 'log_B_12', 'log_B_13', 'log_B_21', 'log_B_22', 'log_B_23', 'log_B_24', 'log_B_26', 'log_B_27', 'log_B_28', 'log_B_29', 'log_B_3', 'log_B_32', 'log_B_36', 'log_B_4', 'log_B_40', 'log_B_41', 'log_B_42', 'log_B_5', 'log_B_9', 'log_D_106', 'log_D_107', 'log_D_108', 'log_D_109', 'log_D_113', 'log_D_115', 'log_D_118', 'log_D_119', 'log_D_123', 'log_D_125', 'log_D_131', 'log_D_133', 'log_D_135', 'log_D_136', 'log_D_137', 'log_D_138', 'log_D_140', 'log_D_39', 'log_D_41', 'log_D_43', 'log_D_44', 'log_D_45', 'log_D_49', 'log_D_51']

print(float_feats)
print(log_feats)

['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_32', 'B_33', 'B_36', 'B_37', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D

## define autoencoder-mlp model

In [6]:
#https://pytorch-forecasting.readthedocs.io/en/latest/_modules/pytorch_forecasting/models/mlp/submodules.html#FullyConnectedModule
#https://www.kaggle.com/c/jane-street-market-prediction/discussion/224348
#https://www.kaggle.com/code/gogo827jz/jane-street-supervised-autoencoder-mlp/notebook?scriptVersionId=73762661

import torch
from torch import nn
import numpy as np


class AE_MLP(nn.Module):
    def __init__(
        self,
        input_size: int,
        output_size: int,
        hidden_sizes: list,
        dropouts: list,
    ):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.dropouts = dropouts
        
        #----normalize input data--------------
        self.bn0 = nn.BatchNorm1d(input_size)
        
        #---encoder layer----------------
        self.encoder = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]), 
                                     nn.BatchNorm1d(hidden_sizes[0]), 
                                     nn.SiLU()
                                    )
        #---decoder layer----------------
        self.decoder = nn.Sequential(nn.Dropout(dropouts[0]), 
                                     nn.Linear(hidden_sizes[0], input_size) 
                                    )
        #----AE output layer-------------
        self.ae_out = nn.Sequential(nn.Linear(input_size, hidden_sizes[1]), 
                                    nn.BatchNorm1d(hidden_sizes[1]), 
                                    nn.SiLU(), 
                                    nn.Dropout(dropouts[1]), 
                                    nn.Linear(hidden_sizes[1], output_size),
                                    nn.Sigmoid(), #for binary classification loss function BCELoss
                                    )
        #---MLP--------------------------
                
        # input layer
        size2 = input_size+hidden_sizes[0]
        module_list = [nn.BatchNorm1d(size2), 
                       nn.Dropout(dropouts[2]), 
                       nn.Linear(size2, hidden_sizes[2]), 
                       nn.BatchNorm1d(hidden_sizes[2]), 
                       nn.SiLU(), 
                       nn.Dropout(dropouts[2])]
    
        # hidden layers
        for i in range(3, len(hidden_sizes)):
            module_list.extend([nn.Linear(hidden_sizes[i-1], hidden_sizes[i]), 
                                nn.BatchNorm1d(hidden_sizes[i]), 
                                nn.SiLU(), 
                                nn.Dropout(dropouts[i])]
                              )
        # output layer
        module_list.extend([nn.Linear(hidden_sizes[-1], output_size), 
                           nn.Sigmoid()])

        self.mlp = nn.Sequential(*module_list)
        
    def forward(self, x):
        x0 = self.bn0(x)
        encoder = self.encoder(x0)
        decoder = self.decoder(encoder)
        out_ae = self.ae_out(decoder)
        
        #x0 shape is n*m - n samples, m features
        #encoder is n*k - n samples, k features
        #x1 is n*(k+m) - n samples, (k+m) features
        #if x0 is n*w*m with w as the width for 3d array, the output will be n*w*(k+m)
        x1 = torch.cat((x0, encoder), dim = -1) #
        out = self.mlp(x1)

        return decoder, out_ae, out


In [7]:
import torch
from torch.utils.data import (Dataset, DataLoader)
  

class TS_Data(Dataset):
    
    def __init__(self, X, y): 
        
        features = torch.FloatTensor(X)
        targets = torch.FloatTensor(y)
        
        self.features = features
        self.targets = targets
        
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        
    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        

        x = self.features[idx]
        y = self.targets[idx]
        
        return x, y
    

def load_data(X, y, batch_size, n_workers=0, shuffle=False):
    data = TS_Data(X, y)
    
    loader = DataLoader(data, batch_size=batch_size, num_workers=n_workers, shuffle=shuffle)
    
    return loader

## hyperopt parameters

In [8]:
learn_rates = np.concatenate((np.arange(0.00001, 0.0001, 0.00001),  
                           np.arange(0.0001, 0.001, 0.0001), 
                           np.arange(0.001, 0.01, 0.001), 
                           np.arange(0.01, 0.05, 0.01)
                          ), 
                          axis=0)
hidden_sizes=[32, 48, 64, 96, 128, 256, 448, 512, 896, 1024] 
dropouts = np.round(np.arange(0.001, 0.501, 0.001), 4)

len(learn_rates), len(hidden_sizes), len(dropouts)

(31, 10, 500)

In [9]:
from hyperopt import hp
import numpy as np
space  = { 
             'batch_size': hp.choice('batch_size', [128*16, 128*20, 128*32, 128*40, 128*80,  128*120]),
             'num_epochs':hp.choice('num_epochs', [50, 60, 100, 150, 200]),
             'learning_rate':hp.choice('learning_rate', learn_rates),
             'hidden_size1':hp.choice('hidden_size1', hidden_sizes),
             'hidden_size2':hp.choice('hidden_size2', hidden_sizes),
             'hidden_size3':hp.choice('hidden_size3', hidden_sizes),
             'hidden_size4':hp.choice('hidden_size4', hidden_sizes),
             'hidden_size5':hp.choice('hidden_size5', hidden_sizes),
             'hidden_size6':hp.choice('hidden_size6', hidden_sizes),
             'dropout1':  hp.choice('dropout1', dropouts), 
             'dropout2':  hp.choice('dropout2', dropouts), 
             'dropout3':  hp.choice('dropout3', dropouts), 
             'dropout4':  hp.choice('dropout4', dropouts), 
             'dropout5':  hp.choice('dropout5', dropouts), 
             'dropout6':  hp.choice('dropout6', dropouts), 
    
            }                  

### Train

In [10]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [11]:
x_cols = float_feats + cat_feats + log_feats
len(x_cols)

269

In [25]:
feats = ['P_2', 'B_20', 'R_1', 'D_46', 'log_B_26', 'log_B_9', 'B_33', 'log_B_23', 'D_39', 'S_3', 'D_43', 'B_18', 'log_B_22', 'R_8', 'log_D_51', 'R_4', 'B_9', 'B_1', 'B_10']
len(feats)

19

In [ ]:
X = df[feats]
y = df[['target']]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 273 entries, customer_ID to target
dtypes: float32(267), int32(2), int64(1), object(3)
memory usage: 488.4+ MB


In [15]:
del df
gc.collect()

21

In [27]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
skf.get_n_splits(X, y)

5

In [28]:
print(skf)

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
TRAIN: [     0      1      2 ... 458910 458911 458912] TEST: [     4      7     19 ... 458899 458902 458906]
TRAIN: [     2      3      4 ... 458909 458910 458911] TEST: [     0      1      5 ... 458907 458908 458912]
TRAIN: [     0      1      2 ... 458909 458910 458912] TEST: [    13     17     26 ... 458901 458903 458911]
TRAIN: [     0      1      3 ... 458909 458911 458912] TEST: [     2     10     11 ... 458898 458905 458910]
TRAIN: [     0      1      2 ... 458910 458911 458912] TEST: [     3      6     15 ... 458897 458904 458909]


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [19]:
import numpy as np
import torch
from tqdm import tqdm
import torch.optim as optim
from torch.nn import CrossEntropyLoss, MSELoss, BCELoss

In [20]:
loss_dict = []

def score(params):
    pl.seed_everything(1)
    print(params)
    
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    h_sizes = [params[f'hidden_size{i}'] for i in range(1,7)]
    drop_list = [params[f'dropout{i}'] for i in range(1,7)]


    losses = []
    
    for train_index, test_index in skf.split(X, y):
        
        #----start: data prep-------------------------------------
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #----end: data prep-------------------------------------
        
#         print(X_train.shape, X_test.shape)
        
        
        minmax_scaler = MinMaxScaler()
        minmax_scaler.fit(X_train)
        # minmax_scaler.fit_transform(X_train[x_cols])

        train_loader = load_data(minmax_scaler.transform(X_train), y_train['target'].values, 
                                 batch_size=batch_size, n_workers=0, shuffle=False)

        test_loader = load_data(minmax_scaler.transform(X_test), y_test['target'].values, 
                                 batch_size=batch_size, n_workers=0, shuffle=False)
        #----end: data prep-------------------------------------


        model = AE_MLP(input_size=len(feats), output_size=1, 
                       hidden_sizes=h_sizes, 
                       dropouts = drop_list)

        model = model.to(device)

        # optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)  
        optimizer = torch.optim.RMSprop([
                {'params': model.encoder.parameters()},
                {'params': model.decoder.parameters()},
                {'params': model.ae_out.parameters()},
                {'params': model.mlp.parameters()},
            ], lr=learning_rate)


        decoder_loss = MSELoss()
        out_ae_loss = BCELoss()
        out_loss = BCELoss()


        #------train models--------------------------
        for epoch in range(num_epochs):
            model.train()
            for batch_idx, (features, targets) in enumerate(train_loader):

                features = features.to(device)
                targets = targets.to(device)

                ### FORWARD AND BACK PROP
                decoder, out_ae, out = model(features)

                decoder_cost = decoder_loss(decoder, features)
                out_ae_cost = out_ae_loss(out_ae.squeeze(), targets)  
                out_cost = out_loss(out.squeeze(), targets) #squeeze the n_samples*1 2d array to 1d array of n_samples
                total_cost = (decoder_cost + out_ae_cost + out_cost)/3

                optimizer.zero_grad()

                total_cost.backward()

                ### UPDATE MODEL PARAMETERS
                optimizer.step()

        #-----eval models-------------------------------
        model.eval()

        y_preds = []
        y_trues = []
        with torch.no_grad():
            for features, targets in test_loader:
                features = features.to(device)
                targets = targets.to(device)
                _, _, outputs = model(features)
                y_preds.extend(outputs.squeeze().cpu().numpy())
                y_trues.extend(targets.squeeze().cpu().numpy())

        #-----start: train mlp---------------------------------------
        
        #-----end: train mlp---------------------------------------
#         loss = roc_auc_score(y_trues, y_preds)
        loss = amex_metric(y_test, 
                           pd.DataFrame(data={'prediction': y_preds}))
        
        losses.append(loss)
        
        
    loss = np.mean(losses)
    print(loss)
    loss_dict.append({'params': params, 'losses': losses, 'mean_loss': loss})
    
    return {'loss': -loss, 'status': STATUS_OK}

In [21]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, anneal, rand
from functools import partial
def optimize(space, evals, cores, trials, optimizer=tpe.suggest, random_state=1234, n_startup_jobs=10):
    algo = partial(optimizer, n_startup_jobs=n_startup_jobs)
    best = fmin(score, space, algo=algo, max_evals=evals, trials = trials)
    print(best)
    return best

In [22]:
cores = 4
n=500
verbose = False
trials = Trials()

In [ ]:
best_param = optimize(space,
                      evals = n,
                      optimizer=tpe.suggest,
                      cores = cores,
                      trials = trials, random_state=1234, 
                      n_startup_jobs=10)